In [ ]:
# Purpose of this notebook: Basics of running GP attack
# Outputs: Lists of candidate perturbations
# Next Steps: Take these outputs and evaluate them to find the best perturbations, measure performance, etc.

# Import statements and initial defines

In [ ]:
# Don't stress about the future...
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning)

# Basic imports
import sys
import os
import imp
import re
import pickle
import math
import copy
import random
from pathlib import Path
import numpy as np

# Audio inputs
import librosa
import soundfile as sf
import sounddevice as sd

# ML Imports
import tensorflow as tf
import tensorflow.keras as keras

In [ ]:
# Setup to import custom modules
if not f'{os.getcwd()}/../scripts' in sys.path:
    sys.path.append(f'{os.getcwd()}/../scripts')

In [ ]:
# Get text transcripts from audio files
import transcription
#_ = imp.reload(transcription)

# Load audio datasets and get info based upon filenames
import audiodatasets
#_ = imp.reload(audiodatasets)

# Ease of use with my dictionaries
import dictmgmt
#_ = imp.reload(dictmgmt)

# Other miscellaneous utilities (like the timer)
import misc
#_ = imp.reload(misc)


In [ ]:
# Load classifier utilities
import classifiers
#_ = imp.reload(classifiers)

In [ ]:
import gp
#_ = imp.reload(gp)

In [ ]:
# Define some basic variables for this runtime
jar = f'{os.getcwd()}/../pickles'
model_dir = f'{os.getcwd()}/../models'
data_dir = f'{os.getcwd()}/../data'

# Dataset management

In [ ]:
# Load the datasets; "pure" is digital data, "RECORDED" was played and recorded in our target environment E 
ravdess_files = audiodatasets.list_audio_files(os.path.join(data_dir, 'fixed_ravdess'))
ravdess_files_acoustic = audiodatasets.list_audio_files(os.path.join(data_dir, 'fixed_ravdess_acoustic'))
ravdess_speaker_ids = {audiodatasets.get_speaker_id(key) for key in ravdess_files.keys()}
ravdess_sentence_ids = {audiodatasets.get_sentence_id(key) for key in ravdess_files.keys()}

tess_files = audiodatasets.list_audio_files(os.path.join(data_dir, 'fixed_tess'))
tess_files_acoustic = audiodatasets.list_audio_files(os.path.join(data_dir, 'fixed_tess_avery_1ft'))
tess_files_acoustic2 = audiodatasets.list_audio_files(os.path.join(data_dir, 'fixed_tess_avery_3ft'))
tess_files_acoustic3 = audiodatasets.list_audio_files(os.path.join(data_dir, 'fixed_tess_avery_5ft'))
#tess_files_acoustic = audiodatasets.list_audio_files(os.path.join(data_dir, 'fixed_tess_acoustic_1ft'))
#tess_files_acoustic2 = audiodatasets.list_audio_files(os.path.join(data_dir, 'fixed_tess_acoustic_2ft_usbmic50_btspkr84'))
#tess_files_acoustic3 = audiodatasets.list_audio_files(os.path.join(data_dir, 'fixed_tess_acoustic_3ft_usbmic50_btspkr100'))
tess_files_acoustic = {**tess_files_acoustic}
#tess_files_acoustic = {**tess_files_acoustic, **tess_files_acoustic2, **tess_files_acoustic3}

tess_speaker_ids = {audiodatasets.get_speaker_id(key) for key in tess_files.keys()}
tess_sentence_ids = {audiodatasets.get_sentence_id(key) for key in tess_files.keys()}
all_data_files = {"PURE": {**ravdess_files, **tess_files}, "RECORDED": {**ravdess_files_acoustic, **tess_files_acoustic}} 

# save the data file structure
pickle.dump(all_data_files, open(os.path.join(jar, 'all_data_files.pickle'), 'wb'))

# Run these files through our classifier(s) to see what we can correctly classify

In [ ]:
misc.resetTimer()

emo_classifiers = {"PURE": classifiers.models["SIMPLE_PURE"], "RECORDED": classifiers.models["SIMPLE_RECORDED"]}
feature_extractors = {"PURE": classifiers.feature_extractors["SIMPLE_PURE"], "RECORDED": classifiers.feature_extractors["SIMPLE_RECORDED"]}
label_getters = {"PURE": classifiers.label_getters["SIMPLE_PURE"], "RECORDED": classifiers.label_getters["SIMPLE_RECORDED"]}

valid_classification = {}
original_softmax_values = {}

if os.path.exists(os.path.join(jar, 'valid_classification.pickle')) and os.path.exists(os.path.join(jar, 'original_softmax_values.pickle')):
    valid_classification = pickle.load(open(os.path.join(jar, 'valid_classification.pickle'), 'rb'))
    original_softmax_values = pickle.load(open(os.path.join(jar, 'original_softmax_values.pickle'), 'rb'))
else:
    for key in all_data_files.keys():
        dataset = all_data_files[key]
        emo_model = emo_classifiers[key]
        feature_extractor = feature_extractors[key]
        label_getter = label_getters[key]

        X_vals, y_vals, featureindices = feature_extractor(dataset)
        predictions = emo_model.predict(X_vals) 

        for sample_key in featureindices.keys():
            actual, predicted = label_getter(y_vals, predictions, featureindices[sample_key])
            dictmgmt.dictset(original_softmax_values, [key, sample_key], copy.deepcopy(predictions[featureindices[sample_key]]))

            if actual == predicted:
                dictmgmt.dictset(valid_classification, [key, sample_key], all_data_files[key][sample_key])
    pickle.dump(valid_classification, open(os.path.join(jar, 'valid_classification.pickle'), 'wb'))
    pickle.dump(original_softmax_values, open(os.path.join(jar, 'original_softmax_values.pickle'), 'wb'))

misc.elapsedTime()

# Run these files through our transcription service to see what we can correctly transcribe

In [ ]:
misc.resetTimer()

all_transcripts = {}
all_confidences = {}

if os.path.exists(os.path.join(jar, 'all_transcripts.pickle')) and os.path.exists(os.path.join(jar, 'all_confidences.pickle')):
    all_transcripts = pickle.load(open(os.path.join(jar, 'all_transcripts.pickle'), 'rb'))
    all_confidences = pickle.load(open(os.path.join(jar, 'all_confidences.pickle'), 'rb'))
else:
    for key in all_data_files.keys():
        transcripts, confidences = transcription.extract_text(all_data_files[key])
        all_transcripts[key] = transcripts
        all_confidences[key] = confidences

    pickle.dump(all_transcripts, open(os.path.join(jar, 'all_transcripts.pickle'), 'wb'))
    pickle.dump(all_confidences, open(os.path.join(jar, 'all_confidences.pickle'), 'wb'))
    
misc.elapsedTime()

In [ ]:
misc.resetTimer()

# Next, validate the transcripts and drop everything else
valid_transcripts = {}
valid_confidences = {}

if os.path.exists(os.path.join(jar, 'valid_transcripts.pickle')) and os.path.exists(os.path.join(jar, 'valid_confidences.pickle')):
    valid_transcripts = pickle.load(open(os.path.join(jar, 'valid_transcripts.pickle'), 'rb'))
    valid_confidences = pickle.load(open(os.path.join(jar, 'valid_confidences.pickle'), 'rb'))
else:
    for key in all_data_files.keys():
        _, transcripts, confidences = transcription.remove_transcription_errors(all_data_files[key], all_transcripts[key], all_confidences[key])
        valid_transcripts[key] = transcripts
        valid_confidences[key] = confidences

    pickle.dump(valid_transcripts, open(os.path.join(jar, 'valid_transcripts.pickle'), 'wb'))
    pickle.dump(valid_confidences, open(os.path.join(jar, 'valid_confidences.pickle'), 'wb'))
    
misc.elapsedTime()

In [ ]:
pickle.dump(valid_classification, open(os.path.join(jar, 'valid_classification.pickle'), 'wb'))
pickle.dump(original_softmax_values, open(os.path.join(jar, 'original_softmax_values.pickle'), 'wb'))
pickle.dump(all_transcripts, open(os.path.join(jar, 'all_transcripts.pickle'), 'wb'))
pickle.dump(all_confidences, open(os.path.join(jar, 'all_confidences.pickle'), 'wb'))

In [ ]:
misc.alert_me()

# Now, we just need to take the intersection of the validly classified and validly transcribed; this is our dataset

In [ ]:
evasive_todo = {}

for key in all_data_files.keys():
    for sample_key in all_data_files[key].keys():
        if sample_key in valid_transcripts[key] and sample_key in valid_classification[key].keys():
            dictmgmt.dictset(evasive_todo, [key, sample_key], all_data_files[key][sample_key])

    a = len(all_data_files[key])
    b = len(valid_classification[key])
    c = len(valid_transcripts[key])
    d = len(evasive_todo[key])
    print(f"Original {key} Dataset: {a} | Correctly Classified: {b} | Correctly Transcribed: {c} | Intersection: {d}")

# Finally, load any autoencoders for use estimating the target environment E

In [ ]:
def custom_loss(y_true, y_pred, sample_weight=None, **kwargs):
    foo = tf.real(tf.signal.fft(tf.complex(y_true, 0.0)))
    foo1 = tf.real(tf.signal.fft(tf.complex(y_pred, 0.0)))
    return keras.losses.mean_squared_error(foo, foo1)

# Update: Removed autoencoders because they did not significantly improve performance

#autoencoders = { "MSE": keras.models.load_model(f'{model_dir}/autoencoder.h5'),
#                 "MSE-FFT": keras.models.load_model(f'{model_dir}/autoencoder_customloss.h5', custom_objects={"custom_loss":custom_loss})
#               }
#autoencoders = { "MSE": keras.models.load_model(f'{model_dir}/autoencoder.h5'),
#                 "MSE-FFT": keras.models.load_model(f'../data/autoencoder_custom_avery.h5', custom_objects={"custom_loss":custom_loss})
#               }

# Now that we've got the dataset and the parts for our validation framework, setup the GP environment

In [ ]:
# Define some baseline perturbation characteristics; in general, should not change these between runs
min_freq=100
max_freq=4000 
min_duration=2.5
max_duration=4.0
min_offset=0.0 
max_offset=0.5
min_muzzle=25
max_muzzle=150

In [ ]:
# First, I need to define my fitness function; this leverages a lot of global variables b/c of how DEAP is written
# (Technically, DEAP supports more than one, but this is simpler and works more consistently)
# At a high level:
# def evaluate():
#     Sample the training dataset
#     Turn the individual candidate into a perturbation
#     Apply the perturbation to the sample
#     score = text_transcription_goodness * classification_fool_score
#     return score

In [ ]:
# Applies a single perturbation to a single file
def apply_perturbation(individual, audio_key, filelist, temp_dir='/tmp/bpt', max_duration=4.0, sample_rate=48000, autoencoder_key=None):

    # Everything is file based...make sure that the temp directory is there
    Path(temp_dir).mkdir(parents=True, exist_ok=True)
    
    x, sr = librosa.load(filelist[audio_key], duration=max_duration, sr=sample_rate)

    new_x = x
    for freq,duration,offset,muzzle in zip(individual[::4], individual[1::4], individual[2::4], individual[3::4]):

        # Adjust normalized values
        freq = freq * (max_freq - min_freq) + min_freq
        duration = duration * (max_duration - min_duration) + min_duration
        offset = offset * (max_offset - min_offset) + min_offset
        muzzle = muzzle * (max_muzzle - min_muzzle) + min_muzzle
        
        tone = librosa.tone(freq, duration=duration, sr=sample_rate)
        tone = tone/muzzle
        int_off = int(sr * offset)
        
        # Called if we are running acoustically (i.e. - using the autoencoder to simulate the room's acoustics)
#        if not autoencoder_key is None:
#            tone = librosa.util.fix_length(tone, int(max_duration*sample_rate))
            
            # Need to resample down and up for the autoencoder (ae hardcoded to sr=10000)
#            tone = librosa.resample(tone, orig_sr=sample_rate, target_sr=10000)
#            acoustic_tone = autoencoders[autoencoder_key].predict(np.array([np.reshape(tone, (np.shape(tone)[0], 1))]))[0]
#            acoustic_tone = librosa.resample(tone, orig_sr=10000, target_sr=sample_rate)
#            tone=acoustic_tone
        
        for indx in range(int_off, min(len(tone)+int_off, len(new_x))):
            new_x[indx] += tone[indx-int_off]

    sf.write(f'{temp_dir}/{audio_key}', new_x, sample_rate, )
    return(f'{temp_dir}/{audio_key}')

In [ ]:
# One important attribute is the impact of tones on the extracted text; returns a value in [0, 1]
# This goes to 0 if there are ANY new transcription errors
# The score is decreased linearly with any impacts to confidence
def measure_text_extract_goodness(new_transcripts, new_confidences, orig_confidences):
    retval = 1.0
    num_samples = len(new_transcripts)
    
    for key, text in new_transcripts.items():
        if not transcription.valid_transcript(text):
            retval = 0.0
            break
        
        try:
            if new_confidences[key] < orig_confidences[key]:
                retval -= (orig_confidences[key] - new_confidences[key]) / num_samples
        except KeyError:
            # Skip this one if there is a KeyError (shouldn't happen)
            print(f'Key Error: {key}')
            
    return retval

In [ ]:
# Another important factor is the classifier score; if this modification pushes me towards a different class, then we get credit for that
# Modified files is a dictionary with key=filename (same is the original) and value=path to possibly evasive file
# Impacts to score will be compared with values in the  original_softmax_results dictionary generated earlier

def measure_classification_degradation( 
                                        classifier, # classifier being used
                                        feature_extractor, # feature extractor for this classifier
                                        label_getter, # knows how to parse the results
                                        original_softmax, # dict containing softmax values for original files
                                        modified_files, # dict containing files that we have modified to fool the classifier 
                                        misclassification_bonus=50, # Bonus given to score if we have an actual misclassification 
                                        target_class=None
                                      ):

    score = 0.0
    
    X_vals, y_vals, featureindices = feature_extractor(modified_files)
    predictions = classifier.predict(X_vals) 

    for sample_key in featureindices.keys():
        actual, predicted = label_getter(y_vals, predictions, featureindices[sample_key])

        
        
        # Bonus for misclassifications
        if target_class is None: # untargeted evason

            score += max(original_softmax[sample_key][actual]-predictions[featureindices[sample_key]][actual], 0)
            
            # Bonus
            if actual != predicted:
                score += misclassification_bonus
        else: # targeted evason

            score += max(predictions[featureindices[sample_key]][target_class] - original_softmax[sample_key][target_class], 0)

            # Bonus
            if predicted == target_class:
                score += misclassification_bonus
    return score


In [ ]:
# Finally, we pull together all of these measures into a single fitness measure
# DEAP does support multiple, independent fitness measures but they were not working for me
def evaluation(individual):
    
    # The globals used by this function are stored in a dictionary
    global evaluation_dictionary
    
    evaluation_file_list = evaluation_dictionary["Evaluation File List"]
    evaluation_sample_size = evaluation_dictionary["Evaluation Sample Size"]
    original_transcription_confidences = evaluation_dictionary["Original Transcription Confidences"]
    evaluation_autoencoder_key = evaluation_dictionary["Evaluation Autoencoder Key"]
    evaluation_classifier_modality = evaluation_dictionary["Evaluation Classifier Modality"] # PURE|RECORDED
    target_class = evaluation_dictionary["Target Class"]
    original_softmax = evaluation_dictionary["Original Softmax Values"]
    
    # Derive a few from the classifier modality
    emo_classifier = emo_classifiers[evaluation_classifier_modality]
    feature_extractor = feature_extractors[evaluation_classifier_modality]
    label_getter = label_getters[evaluation_classifier_modality]

    # Take a random sample and generate modified versions of these files using individual's perturbation
    sample_keys = random.sample(list(evaluation_file_list), min(evaluation_sample_size, len(evaluation_file_list)))

    modified_files = { key:apply_perturbation(individual=individual, audio_key=key, filelist=evaluation_file_list, autoencoder_key=evaluation_autoencoder_key) \
                      for key in sample_keys 
                     }

    # Now, assess the fitness of this individual
    fitness_score = 0.0

    if len(modified_files) > 0:
        classifier_fool_score = measure_classification_degradation( classifier=emo_classifier, 
                                                                    feature_extractor=feature_extractor,
                                                                    label_getter=label_getter,
                                                                    original_softmax=original_softmax,
                                                                    modified_files=modified_files,
                                                                    target_class=target_class
                                                                  )
        text_extract_score = 0.0

        # Only waste time on the text extraction if we are making some progress with the classifier
        if classifier_fool_score > 0.0:

            # Adding some retires in here becaue the VOSK stuff is a little buggy (with the model that I'm using)
            eval_transcripts = {}
            eval_confidences = {}

            # 10 retries should be enough
            for _ in range(10):
                if len(modified_files) == 0:
                    break

                new_transcripts, new_confidences = transcription.extract_text(modified_files)

                remove_me = []
                for key in new_transcripts.keys():
                    if new_confidences[key] != 0.0:
                        eval_transcripts[key] = new_transcripts[key]
                        eval_confidences[key] = new_confidences[key]
                        remove_me.append(key)

                for key in remove_me:
                    del modified_files[key]

            text_extract_score = measure_text_extract_goodness(new_transcripts, new_confidences, original_transcription_confidences)

        fitness_score = text_extract_score*classifier_fool_score

    return fitness_score,

In [ ]:
# Finally, we initialize the GP environment and then we are ready to start evaluations
gp.initialize_gp_environment(fitness_function=evaluation)

In [ ]:
misc.alert_me()

# EXPERIMENTS

In [ ]:
# EXPERIMENT 0: 
#    - Part of rewrite; let's just build a few EONs (new AAP name) based upon different group deographics

# Generate based upon gender; RAVDESS ONLY
misc.resetTimer()

groups = {"male": [1, 3, 5, 7, 9], "female": [0, 2, 4, 6, 8], "2male3female": [11, 13, 14, 16, 18], "3male2female": [11, 13, 15, 16, 18]}

for gender in groups:
    if gender == "male" or gender == "female":
        continue
    
    evaluation_dictionary = {}
    evaluation_dictionary["Evaluation File List"] = {

        sample_key:evasive_todo["PURE"][sample_key] for sample_key in evasive_todo["PURE"] \
            if audiodatasets.get_sentence_id(sample_key) in ravdess_sentence_ids and \
               audiodatasets.get_speaker_id(sample_key) in groups[gender]

    }
    evaluation_dictionary["Evaluation Sample Size"] = 40
    evaluation_dictionary["Original Transcription Confidences"] = valid_confidences["PURE"]
    evaluation_dictionary["Evaluation Autoencoder Key"] = None
    evaluation_dictionary["Evaluation Classifier Modality"] = "PURE"
    evaluation_dictionary["Target Class"] = None
    evaluation_dictionary["Original Softmax Values"] = original_softmax_values["PURE"]
    gp.initialize_gp_environment(fitness_function=evaluation)

    populations,_ = gp.run_gp(number_of_generations=40)

    pops = []
    for x in populations:
        pops.append([])
        for y in x:
            pops[-1].append(list(y))

    populations = pops
    
    pops_fn = f"populations_RAVDESS40_GENDER_{gender}.pickle"
    pickle.dump(populations, open(f"{jar}/{pops_fn}", 'wb'))
    print(f"Completed {gender}")
    misc.elapsedTime()

In [ ]:
# Experiment 1: 
#    - Train on RAVDESS data for 40 generations
#    - Train on TESS data for an additional 10 generations
#    - TESS we do a 30/30/140 split based upon phrases from the dataset
#    - Training and Testing use the MSE autoencoder
#    - Evaluation will be done elsewhere (will require playing the perturbations in environment E, recording, etc.)

In [ ]:
misc.resetTimer()

# Part 1: Train on RAVDESS for 40 generations

# Setup the evaluation environment
evaluation_dictionary = {}
evaluation_dictionary["Evaluation File List"] = { sample_key:evasive_todo["PURE"][sample_key] for sample_key in evasive_todo["PURE"] \
                                                 if audiodatasets.get_sentence_id(sample_key) in ravdess_sentence_ids } 
evaluation_dictionary["Evaluation Sample Size"] = 40
#evaluation_dictionary["Evaluation Sample Size"] = 20

# We are starting with a pretrained model, so we need to go w/o any info about the target environment E
evaluation_dictionary["Original Transcription Confidences"] = valid_confidences["PURE"]
evaluation_dictionary["Evaluation Autoencoder Key"] = None
evaluation_dictionary["Evaluation Classifier Modality"] = "PURE"
evaluation_dictionary["Target Class"] = None
evaluation_dictionary["Original Softmax Values"] = original_softmax_values["PURE"]

gp.initialize_gp_environment(fitness_function=evaluation)

# Load the populations from a RAVDESS run; 
# If I didn't have this pickle, I would run:
#   evaluation_dictionary["Evaluation File List"] = { key:evasive_todo["RECORDED"][key] for key in evasive_todo["RECORDED"] \
#                                                     if audiodatasets.get_sentence_id(sample_key) in ravdess_sentence_ids } 
print("\n=======================================================================================")
print(f"RAVDESS/TESS: RAVDESS Digital run")
print("=======================================================================================\n")

#ravdess_populations, _ = gp.run_gp(starting_population=None, number_of_generations=40)
#pickle.dump(ravdess_populations, open(f"{jar}/populations_START_RAVDESS40_PLUS_TESS_10GEN.pickle", 'wb'))
ravdess_populations = pickle.load(open(f'{jar}/populations_RAVDESS_UNTARGETED_CONTINUED.pickle', 'rb'))

print(f"Completed.")
misc.elapsedTime()

# Part 2: Train on a subset of the TESS dataset
autoencoder_keys = ['PURE']
#autoencoder_keys = ['PURE', 'MSE', 'MSE-FFT']
#splits = [80, 80, 80]
splits = [100, 100, 100]

for run_indx in range(len(autoencoder_keys)):

    if autoencoder_keys[run_indx] == 'PURE':
        ae_id = None
    else:
        ae_id = autoencoder_keys[run_indx]
    ss = splits[run_indx]//2
        
    for trial_indx in range(1,3):
        print("\n=======================================================================================")
        print(f"RAVDESS/TESS: Splits {ss}_{ss}_{200-ss*2}, Autoencoder: {autoencoder_keys[run_indx]}, Trial {trial_indx+1} of 2")
        print("=======================================================================================\n")

#        pops_fn = f"populations_DIGITAL_{trial_indx:02}_ae_{autoencoder_keys[run_indx]}_samples_{ss}_{ss}_{200-ss*2}_RAVDESS40_PLUS_TESS_10GEN.pickle"
#        samples_fn = f"samples_DIGITAL_{trial_indx:02}_ae_{autoencoder_keys[run_indx]}_samples_{ss}_{ss}_{200-ss*2}_RAVDESS40_PLUS_TESS_10GEN.pickle"
        
        pops_fn = f"populations_{trial_indx:02}_ae_{autoencoder_keys[run_indx]}_samples_{ss}_{ss}_{200-ss*2}_RAVDESS40_PLUS_TESS_MULTDIST_AVERY_10GEN.pickle"
        samples_fn = f"samples_{trial_indx:02}_ae_{autoencoder_keys[run_indx]}_samples_{ss}_{ss}_{200-ss*2}_RAVDESS40_PLUS_TESS_MULTDIST_AVERY_10GEN.pickle"
        
        if not os.path.exists(f"{jar}/{pops_fn}"):
            sample = random.sample(list(tess_sentence_ids), 60)
#            train_sample = { sample_key:evasive_todo["PURE"][sample_key] for sample_key in evasive_todo["PURE"] \
#                                                             if audiodatasets.get_sentence_id(sample_key) in sample[:ss] } 
#            test_sample = { sample_key:evasive_todo["PURE"][sample_key] for sample_key in evasive_todo["PURE"] \
#                                                             if audiodatasets.get_sentence_id(sample_key) in sample[ss:] } 
#            eval_sample = { sample_key:evasive_todo["PURE"][sample_key] for sample_key in evasive_todo["PURE"] \
#                                                             if not audiodatasets.get_sentence_id(sample_key) in sample } 

            train_sample = { sample_key:evasive_todo["RECORDED"][sample_key] for sample_key in evasive_todo["RECORDED"] \
                                                             if audiodatasets.get_sentence_id(sample_key) in sample[:ss] } 
            test_sample = { sample_key:evasive_todo["RECORDED"][sample_key] for sample_key in evasive_todo["RECORDED"] \
                                                             if audiodatasets.get_sentence_id(sample_key) in sample[ss:] } 
            eval_sample = { sample_key:evasive_todo["RECORDED"][sample_key] for sample_key in evasive_todo["RECORDED"] \
                                                             if not audiodatasets.get_sentence_id(sample_key) in sample } 

            pickle.dump( {"TRAIN": train_sample, "TEST":test_sample, "EVAL": eval_sample}, open(f"{jar}/{samples_fn}", 'wb'))

            evaluation_dictionary["Evaluation File List"] = train_sample
            evaluation_dictionary["Evaluation Autoencoder Key"] = ae_id
            evaluation_dictionary["Original Transcription Confidences"] = valid_confidences["RECORDED"]
            evaluation_dictionary["Evaluation Classifier Modality"] = "RECORDED"
            evaluation_dictionary["Original Softmax Values"] = original_softmax_values["RECORDED"]

            populations,_ = gp.run_gp(starting_population=ravdess_populations[-1], number_of_generations=10)

            pops = []
            for x in populations:
                pops.append([])
                for y in x:
                    pops[-1].append(list(y))

            populations = pops
            pickle.dump( populations, open(f"{jar}/{pops_fn}", 'wb'))

        print(f"Completed trial {trial_indx}")
        misc.elapsedTime()
    
# Part 3: Test

In [ ]:
misc.alert_me()

In [ ]:
# Personalized perturbations
len(evaluation_dictionary['Evaluation File List'])

In [ ]:
misc.resetTimer()

# Part 1: Train on RAVDESS for 40 generations

# Setup the evaluation environment
evaluation_dictionary = {}
evaluation_dictionary["Evaluation File List"] = { sample_key:evasive_todo["PURE"][sample_key] for sample_key in evasive_todo["PURE"] \
                                                 if audiodatasets.get_sentence_id(sample_key) in ravdess_sentence_ids } 
evaluation_dictionary["Evaluation Sample Size"] = 40
#evaluation_dictionary["Evaluation Sample Size"] = 20

# We are starting with a pretrained model, so we need to go w/o any info about the target environment E
evaluation_dictionary["Original Transcription Confidences"] = valid_confidences["PURE"]
evaluation_dictionary["Evaluation Autoencoder Key"] = None
evaluation_dictionary["Evaluation Classifier Modality"] = "PURE"
evaluation_dictionary["Target Class"] = None
evaluation_dictionary["Original Softmax Values"] = original_softmax_values["PURE"]

gp.initialize_gp_environment(fitness_function=evaluation)

# Load the populations from a RAVDESS run; 
# If I didn't have this pickle, I would run:
#   evaluation_dictionary["Evaluation File List"] = { key:evasive_todo["RECORDED"][key] for key in evasive_todo["RECORDED"] \
#                                                     if audiodatasets.get_sentence_id(sample_key) in ravdess_sentence_ids } 
print("\n=======================================================================================")
print(f"RAVDESS/TESS: RAVDESS Digital run")
print("=======================================================================================\n")

#ravdess_populations, _ = gp.run_gp(starting_population=None, number_of_generations=40)
#pickle.dump([x for x in [y for y in ravdess_populations[x]]], open(f"{jar}/populations_START_RAVDESS40_diff_params_PLUS_TESS_10GEN.pickle", 'wb'))
#ravdess_populations = pickle.load(open(f'{jar}/populations_RAVDESS_UNTARGETED_CONTINUED.pickle', 'rb'))

print(f"Completed.")
misc.elapsedTime()

# Do the case with all users to baseline
#for trial_indx in range(0,5):
#    print("\n=======================================================================================")
#    print(f"RAVDESS/TESS Personalized: Speaker ID ALL, Trial {trial_indx+1} of 5")
#    print("=======================================================================================\n")

#    pops_fn = f"populations_PERSONAL_diff_params_{trial_indx:02}_speakerid_ALL_RAVDESS40_PLUS_TESS_10GEN.pickle"
#    samples_fn = f"samples_PERSONAL_diff_params_{trial_indx:02}_speakerid_ALL_RAVDESS40_PLUS_TESS_10GEN.pickle"

#    if not os.path.exists(f"{jar}/{pops_fn}"):
#        total_set = { sample_key:evasive_todo["PURE"][sample_key] for sample_key in evasive_todo["PURE"] \
#                                                         if audiodatasets.get_speaker_id(sample_key) in tess_speaker_ids }
#        total_set = { key:total_set[key] for key in random.sample(list(total_set.keys()), 206) }

#        train_keys = random.sample(list(total_set.keys()), int(len(total_set)*.1))
#        test_keys = random.sample([k for k in total_set.keys() if k not in train_keys], int(len(total_set)*.1))
#        eval_keys = [k for k in total_set.keys() if k not in train_keys and k not in test_keys]

#        train_sample = { sample_key:evasive_todo["PURE"][sample_key] for sample_key in train_keys } 
#        test_sample = { sample_key:evasive_todo["PURE"][sample_key] for sample_key in test_keys } 
#        eval_sample = { sample_key:evasive_todo["PURE"][sample_key] for sample_key in eval_keys } 

#        pickle.dump( {"TRAIN": train_sample, "TEST":test_sample, "EVAL": eval_sample}, open(f"{jar}/{samples_fn}", 'wb'))

#        evaluation_dictionary["Evaluation File List"] = train_sample
#        populations,_ = gp.run_gp(starting_population=ravdess_populations[-1], number_of_generations=10)

#        pops = []
#        for x in populations:
#            pops.append([])
#            for y in x:
#                pops[-1].append(list(y))

#        populations = pops
#        pickle.dump( populations, open(f"{jar}/{pops_fn}", 'wb'))

#    print(f"Completed trial {trial_indx}")
#    misc.elapsedTime()



for tess_spkr in tess_speaker_ids:
    
    for trial_indx in range(0,5):
        print("\n=======================================================================================")
        print(f"RAVDESS/TESS Personalized: Speaker ID {tess_spkr}, Trial {trial_indx+1} of 5")
        print("=======================================================================================\n")

        pops_fn = f"populations_PERSONAL_diff_params_{trial_indx:02}_speakerid_{tess_spkr}_RAVDESS40_PLUS_TESS_10GEN.pickle"
        samples_fn = f"samples_PERSONAL_diff_params_{trial_indx:02}_speakerid_{tess_spkr}_RAVDESS40_PLUS_TESS_10GEN.pickle"
        
        if not os.path.exists(f"{jar}/{pops_fn}"):
            total_set = { sample_key:evasive_todo["PURE"][sample_key] for sample_key in evasive_todo["PURE"] \
                                                             if audiodatasets.get_speaker_id(sample_key) == tess_spkr } 

            train_keys = random.sample(list(total_set.keys()), int(len(total_set)*.1))
            test_keys = random.sample([k for k in total_set.keys() if k not in train_keys], int(len(total_set)*.1))
            eval_keys = [k for k in total_set.keys() if k not in train_keys and k not in test_keys]
          
            train_sample = { sample_key:evasive_todo["PURE"][sample_key] for sample_key in train_keys } 
            test_sample = { sample_key:evasive_todo["PURE"][sample_key] for sample_key in test_keys } 
            eval_sample = { sample_key:evasive_todo["PURE"][sample_key] for sample_key in eval_keys } 

            pickle.dump( {"TRAIN": train_sample, "TEST":test_sample, "EVAL": eval_sample}, open(f"{jar}/{samples_fn}", 'wb'))

            evaluation_dictionary["Evaluation File List"] = train_sample
            populations,_ = gp.run_gp(starting_population=ravdess_populations[-1], number_of_generations=10)

            pops = []
            for x in populations:
                pops.append([])
                for y in x:
                    pops[-1].append(list(y))

            populations = pops
            pickle.dump( populations, open(f"{jar}/{pops_fn}", 'wb'))

        print(f"Completed trial {trial_indx}")
        misc.elapsedTime()
    
# Part 3: Test

In [ ]:
ravdess_populations

In [ ]:
##### EXPLANATION-DRIVEN TESTS

In [ ]:
# New experiment - explanation-driven evasion
tone_ranges = [[800, 1000], [533, 734], [1071, 1317]]

# Applies a single perturbation to a single file
def apply_perturbation(individual, audio_key, filelist, temp_dir='/tmp/bpt', max_duration=4.0, sample_rate=48000, autoencoder_key=None):

    # Everything is file based...make sure that the temp directory is there
    Path(temp_dir).mkdir(parents=True, exist_ok=True)
    
    x, sr = librosa.load(filelist[audio_key], duration=max_duration, sr=sample_rate)

    new_x = x

    # NEW FOR EXPLANATION
    tone_indx = 0
    
    for freq,duration,offset,muzzle in zip(individual[::4], individual[1::4], individual[2::4], individual[3::4]):

        # Adjust normalized values

        # NEW FOR EXPLANATION
        freq = freq * (tone_ranges[tone_indx][1] - tone_ranges[tone_indx][0]) + tone_ranges[tone_indx][0]
        tone_indx += 1
                
        duration = duration * (max_duration - min_duration) + min_duration
        offset = offset * (max_offset - min_offset) + min_offset
        muzzle = muzzle * (max_muzzle - min_muzzle) + min_muzzle
        
        tone = librosa.tone(freq, duration=duration, sr=sample_rate)
        tone = tone/muzzle
        int_off = int(sr * offset)
        
        # Called if we are running acoustically (i.e. - using the autoencoder to simulate the room's acoustics)
#        if not autoencoder_key is None:
#            tone = librosa.util.fix_length(tone, int(max_duration*sample_rate))
            
            # Need to resample down and up for the autoencoder (ae hardcoded to sr=10000)
#            tone = librosa.resample(tone, orig_sr=sample_rate, target_sr=10000)
#            acoustic_tone = autoencoders[autoencoder_key].predict(np.array([np.reshape(tone, (np.shape(tone)[0], 1))]))[0]
#            acoustic_tone = librosa.resample(tone, orig_sr=10000, target_sr=sample_rate)
#            tone=acoustic_tone
        
        for indx in range(int_off, min(len(tone)+int_off, len(new_x))):
            new_x[indx] += tone[indx-int_off]

    sf.write(f'{temp_dir}/{audio_key}', new_x, sample_rate, )
    return(f'{temp_dir}/{audio_key}')

In [ ]:
misc.resetTimer()

# Part 1: Train on RAVDESS for 40 generations

# Setup the evaluation environment
evaluation_dictionary = {}
evaluation_dictionary["Evaluation File List"] = { sample_key:evasive_todo["PURE"][sample_key] for sample_key in evasive_todo["PURE"] \
                                                 if audiodatasets.get_sentence_id(sample_key) in ravdess_sentence_ids } 
evaluation_dictionary["Evaluation Sample Size"] = 40

# We are starting with a pretrained model, so we need to go w/o any info about the target environment E
evaluation_dictionary["Original Transcription Confidences"] = valid_confidences["PURE"]
evaluation_dictionary["Evaluation Autoencoder Key"] = None
evaluation_dictionary["Evaluation Classifier Modality"] = "PURE"
evaluation_dictionary["Target Class"] = None
evaluation_dictionary["Original Softmax Values"] = original_softmax_values["PURE"]

gp.initialize_gp_environment(fitness_function=evaluation)

# Load the populations from a RAVDESS run; 
# If I didn't have this pickle, I would run:
#   evaluation_dictionary["Evaluation File List"] = { key:evasive_todo["RECORDED"][key] for key in evasive_todo["RECORDED"] \
#                                                     if audiodatasets.get_sentence_id(sample_key) in ravdess_sentence_ids } 
print("\n=======================================================================================")
print(f"RAVDESS/TESS: RAVDESS Digital run")
print("=======================================================================================\n")

ravdess_populations, _ = gp.run_gp(starting_population=None, number_of_generations=40)
pickle.dump(ravdess_populations, open(f"{jar}/populations_START_RAVDESS40_PLUS_TESS_10GEN_EXPLANATION_DRIVEN.pickle", 'wb'))
#ravdess_populations = pickle.load(open(f'{jar}/populations_START_RAVDESS40_PLUS_TESS_10GEN_EXPLANATION_DRIVEN.pickle', 'rb'))

print(f"Completed.")
misc.elapsedTime()

# Part 2: Train on a subset of the TESS dataset
autoencoder_keys = ['PURE', 'MSE', 'MSE-FFT']
splits = [100, 100, 100]

for run_indx in range(len(autoencoder_keys)):

    if autoencoder_keys[run_indx] == 'PURE':
        ae_id = None
    else:
        ae_id = autoencoder_keys[run_indx]
    ss = splits[run_indx]//2
        
    for trial_indx in range(0,5):
        print("\n=======================================================================================")
        print(f"RAVDESS/TESS: Splits {ss}_{ss}_{200-ss*2}, Autoencoder: {autoencoder_keys[run_indx]}, Trial {trial_indx+1} of 5")
        print("=======================================================================================\n")

        
        pops_fn = f"populations_{trial_indx:02}_ae_{autoencoder_keys[run_indx]}_samples_{ss}_{ss}_{200-ss*2}_RAVDESS40_PLUS_TESS_10GEN_EXPLANATION_DRIVEN.pickle"
        samples_fn = f"samples_{trial_indx:02}_ae_{autoencoder_keys[run_indx]}_samples_{ss}_{ss}_{200-ss*2}_RAVDESS40_PLUS_TESS_10GEN_EXPLANATION_DRIVEN.pickle"
        
        if not os.path.exists(f"{jar}/{pops_fn}"):
            sample = random.sample(list(tess_sentence_ids), 60)

            train_sample = { sample_key:evasive_todo["RECORDED"][sample_key] for sample_key in evasive_todo["RECORDED"] \
                                                             if audiodatasets.get_sentence_id(sample_key) in sample[:ss] and re.match(r'.*[23]ft.*', sample_key) is None} 
            test_sample = { sample_key:evasive_todo["RECORDED"][sample_key] for sample_key in evasive_todo["RECORDED"] \
                                                             if audiodatasets.get_sentence_id(sample_key) in sample[ss:] and re.match(r'.*[23]ft.*', sample_key) is None } 
            eval_sample = { sample_key:evasive_todo["RECORDED"][sample_key] for sample_key in evasive_todo["RECORDED"] \
                                                             if not audiodatasets.get_sentence_id(sample_key) in sample and re.match(r'.*[23]ft.*', sample_key) is None } 

            pickle.dump( {"TRAIN": train_sample, "TEST":test_sample, "EVAL": eval_sample}, open(f"{jar}/{samples_fn}", 'wb'))

            evaluation_dictionary["Evaluation File List"] = train_sample
            evaluation_dictionary["Evaluation Autoencoder Key"] = ae_id
            evaluation_dictionary["Original Transcription Confidences"] = valid_confidences["RECORDED"]
            evaluation_dictionary["Evaluation Classifier Modality"] = "RECORDED"
            evaluation_dictionary["Original Softmax Values"] = original_softmax_values["RECORDED"]

            populations,_ = gp.run_gp(starting_population=ravdess_populations[-1], number_of_generations=10)

            pops = []
            for x in populations:
                pops.append([])
                for y in x:
                    pops[-1].append(list(y))

            populations = pops
            pickle.dump( populations, open(f"{jar}/{pops_fn}", 'wb'))

        print(f"Completed trial {trial_indx}")
        misc.elapsedTime()
    
# Part 3: Test

In [ ]:
# BASELINE TEST...

misc.resetTimer()

for trial_indx in range(0,5):

    starts = [random.randint(0,4000-200), random.randint(0,4000-201), random.randint(0,4000-246)]
    tone_ranges = [[starts[0], starts[0]+200], [starts[1], starts[1]+201], [starts[2], starts[2]+246]]

    # Part 1: Train on RAVDESS for 40 generations

    # Setup the evaluation environment
    evaluation_dictionary = {}
    evaluation_dictionary["Evaluation File List"] = { sample_key:evasive_todo["PURE"][sample_key] for sample_key in evasive_todo["PURE"] \
                                                     if audiodatasets.get_sentence_id(sample_key) in ravdess_sentence_ids } 
    evaluation_dictionary["Evaluation Sample Size"] = 40

    # We are starting with a pretrained model, so we need to go w/o any info about the target environment E
    evaluation_dictionary["Original Transcription Confidences"] = valid_confidences["PURE"]
    evaluation_dictionary["Evaluation Autoencoder Key"] = None
    evaluation_dictionary["Evaluation Classifier Modality"] = "PURE"
    evaluation_dictionary["Target Class"] = None
    evaluation_dictionary["Original Softmax Values"] = original_softmax_values["PURE"]

    gp.initialize_gp_environment(fitness_function=evaluation)

    # Load the populations from a RAVDESS run; 
    # If I didn't have this pickle, I would run:
    #   evaluation_dictionary["Evaluation File List"] = { key:evasive_todo["RECORDED"][key] for key in evasive_todo["RECORDED"] \
    #                                                     if audiodatasets.get_sentence_id(sample_key) in ravdess_sentence_ids } 
    print("\n=======================================================================================")
    print(f"RAVDESS/TESS: RAVDESS Digital run #{trial_indx}")
    print("=======================================================================================\n")

    ravdess_populations, _ = gp.run_gp(starting_population=None, number_of_generations=40)
    pickle.dump(ravdess_populations, open(f"{jar}/populations_START_RAVDESS40_PLUS_TESS_10GEN_EXPLANATION_DRIVEN_BASELINE_{trial_indx}.pickle", 'wb'))
    #ravdess_populations = pickle.load(open(f'{jar}/populations_START_RAVDESS40_PLUS_TESS_10GEN_EXPLANATION_DRIVEN_BASELINE.pickle', 'rb'))

    print(f"Completed.")
    misc.elapsedTime()

    # Part 2: Train on a subset of the TESS dataset
    autoencoder_keys = ['PURE', 'MSE', 'MSE-FFT']
    splits = [100, 100, 100]

    for run_indx in range(len(autoencoder_keys)):

        if autoencoder_keys[run_indx] == 'PURE':
            ae_id = None
        else:
            ae_id = autoencoder_keys[run_indx]
        ss = splits[run_indx]//2
        
        print("\n=======================================================================================")
        print(f"RAVDESS/TESS: Splits {ss}_{ss}_{200-ss*2}, Autoencoder: {autoencoder_keys[run_indx]}, Trial {trial_indx+1} of 5")
        print("=======================================================================================\n")
        
        pops_fn = f"populations_{trial_indx:02}_ae_{autoencoder_keys[run_indx]}_samples_{ss}_{ss}_{200-ss*2}_RAVDESS40_PLUS_TESS_10GEN_EXPLANATION_DRIVEN_BASELINE.pickle"
        samples_fn = f"samples_{trial_indx:02}_ae_{autoencoder_keys[run_indx]}_samples_{ss}_{ss}_{200-ss*2}_RAVDESS40_PLUS_TESS_10GEN_EXPLANATION_DRIVEN_BASELINE.pickle"
        
        if not os.path.exists(f"{jar}/{pops_fn}"):
            sample = random.sample(list(tess_sentence_ids), 60)

            train_sample = { sample_key:evasive_todo["RECORDED"][sample_key] for sample_key in evasive_todo["RECORDED"] \
                                                             if audiodatasets.get_sentence_id(sample_key) in sample[:ss] and re.match(r'.*[23]ft.*', sample_key) is None} 
            test_sample = { sample_key:evasive_todo["RECORDED"][sample_key] for sample_key in evasive_todo["RECORDED"] \
                                                             if audiodatasets.get_sentence_id(sample_key) in sample[ss:] and re.match(r'.*[23]ft.*', sample_key) is None } 
            eval_sample = { sample_key:evasive_todo["RECORDED"][sample_key] for sample_key in evasive_todo["RECORDED"] \
                                                             if not audiodatasets.get_sentence_id(sample_key) in sample and re.match(r'.*[23]ft.*', sample_key) is None } 

            pickle.dump( {"TRAIN": train_sample, "TEST":test_sample, "EVAL": eval_sample}, open(f"{jar}/{samples_fn}", 'wb'))

            evaluation_dictionary["Evaluation File List"] = train_sample
            evaluation_dictionary["Evaluation Autoencoder Key"] = ae_id
            evaluation_dictionary["Original Transcription Confidences"] = valid_confidences["RECORDED"]
            evaluation_dictionary["Evaluation Classifier Modality"] = "RECORDED"
            evaluation_dictionary["Original Softmax Values"] = original_softmax_values["RECORDED"]

            populations,_ = gp.run_gp(starting_population=ravdess_populations[-1], number_of_generations=10)

            pops = []
            for x in populations:
                pops.append([])
                for y in x:
                    pops[-1].append(list(y))

            populations = pops
            pickle.dump( populations, open(f"{jar}/{pops_fn}", 'wb'))

        print(f"Completed trial {trial_indx}")
        misc.elapsedTime()
    
# Part 3: Test

In [ ]:
#######################################
# 20 Sep 2022 - New setup for defense testing
#######################################

misc.resetTimer()

# Part 1: Train on RAVDESS for 40 generations

# Setup the evaluation environment
evaluation_dictionary = {}
evaluation_dictionary["Evaluation File List"] = { sample_key:evasive_todo["PURE"][sample_key] for sample_key in evasive_todo["PURE"] \
                                                 if audiodatasets.get_sentence_id(sample_key) in ravdess_sentence_ids } 
evaluation_dictionary["Evaluation Sample Size"] = 40

# We are starting with a pretrained model, so we need to go w/o any info about the target environment E
evaluation_dictionary["Original Transcription Confidences"] = valid_confidences["PURE"]
evaluation_dictionary["Evaluation Autoencoder Key"] = None
evaluation_dictionary["Evaluation Classifier Modality"] = "PURE"
evaluation_dictionary["Target Class"] = None
evaluation_dictionary["Original Softmax Values"] = original_softmax_values["PURE"]

gp.initialize_gp_environment(fitness_function=evaluation)


# Part 1: Initialize the population on RAVDESS data 
print("\n=======================================================================================")
print(f"RAVDESS/TESS: RAVDESS Digital run")
print("=======================================================================================\n")

ravdess_pickle_fn = os.path.join(jar, 'populations_START_RAVDESS40_PLUS_TESS_10GEN_DEFENSE.pickle')
if os.path.exists(ravdess_pickle_fn):
    ravdess_populations = pickle.load(open(ravdess_pickle_fn, 'rb'))
else:
    ravdess_populations, _ = gp.run_gp(starting_population=None, number_of_generations=40)
    pickle.dump(ravdess_populations, open(ravdess_pickle_fn, 'wb'))

print(f"Completed.")
misc.elapsedTime()

# Part 2: Train on a subset of the TESS dataset
# Build train/test/eval split on TESS data based upon sentence ID (want to ensure that each split has disjoint utterances) 
num_trials = 1

# percentage of stuff in train and test; eval is everything else
trn_tst_eval = [25,25]

for trial_indx in range(num_trials):
    trn_sz = trn_tst_eval[0]
    tst_sz = trn_tst_eval[1]
    eval_sz = 100 - trn_sz - tst_sz
    
    print("\n=======================================================================================")
    print(f"RAVDESS/TESS: Splits {trn_sz}%_{tst_sz}%_{eval_sz}%, Trial {trial_indx+1} of {num_trials}")
    print("=======================================================================================\n")

    # Filenames for saving the details from this run
    pops_fn = f"populations_{trial_indx:02}_samples_{trn_sz}_{tst_sz}_{eval_sz}_RAVDESS40_PLUS_TESS_10GEN_DEFENSE.pickle"
    samples_fn = f"samples_{trial_indx:02}_samples_{trn_sz}_{tst_sz}_{eval_sz}_RAVDESS40_PLUS_TESS_10GEN_DEFENSE.pickle"

    # Skip this run if we've already finished it
    if not os.path.exists(os.path.join(jar, pops_fn)):
        sentence_ids = list(tess_sentence_ids)
        random.shuffle(sentence_ids)
        
        train_len = len(sentence_ids)*trn_sz//100
        test_len = len(sentence_ids)*tst_sz//100
        
        train_sample = { sample_key:evasive_todo["PURE"][sample_key] for sample_key in evasive_todo["PURE"] \
                            if audiodatasets.get_sentence_id(sample_key) in sentence_ids[:train_len] }
        
        test_sample = { sample_key:evasive_todo["PURE"][sample_key] for sample_key in evasive_todo["PURE"] \
                            if audiodatasets.get_sentence_id(sample_key) in sentence_ids[train_len:train_len+test_len] }
        
        eval_sample = { sample_key:evasive_todo["PURE"][sample_key] for sample_key in evasive_todo["PURE"] \
                            if audiodatasets.get_sentence_id(sample_key) in sentence_ids[train_len+test_len:] }
        

        # Save these samples for later eval
        pickle.dump( {"TRAIN": train_sample, "TEST":test_sample, "EVAL": eval_sample}, open(os.path.join(jar, samples_fn), 'wb'))

        # Set the eval environment 
        evaluation_dictionary["Evaluation File List"] = train_sample
        evaluation_dictionary["Evaluation Autoencoder Key"] = "PURE"
        evaluation_dictionary["Original Transcription Confidences"] = valid_confidences["PURE"]
        evaluation_dictionary["Evaluation Classifier Modality"] = "PURE"
        evaluation_dictionary["Original Softmax Values"] = original_softmax_values["PURE"]

        # Iterate for 10 more generations using the TESS training data from above
        populations,_ = gp.run_gp(starting_population=ravdess_populations[-1], number_of_generations=10)

        # Dump the population info for later
        pops = []
        for x in populations:
            pops.append([])
            for y in x:
                pops[-1].append(list(y))

        populations = pops
        pickle.dump( populations, open(os.path.join(jar, pops_fn), 'wb'))

        print(f"Completed trial {trial_indx}")
        misc.elapsedTime()